# 인구 이동
---
입력 예시<br>
2 20 50<br>
50 30<br>
20 40<br>
<br>
출력 예시<br>
1<br>
<br><br>
입력 예시<br>
2 40 50<br>
50 30<br>
20 40<br>
<br>
출력 예시<br>
0<br>
<br><br>
입력 예시<br>
3 5 10<br>
10 15 20<br>
20 30 25<br>
40 22 10<br>
<br>
출력 예시<br>
2<br>
<br><br>
입력 예시<br>
4 10 50<br>
10 100 20 90<br>
80 100 60 70<br>
70 20 30 40<br>
50 20 100 10<br>
<br>
출력 예시<br>
3<br>
<br><br>

In [1]:
# dfs 이용

n, l, r = map(int,input().split())
A = [] # 원래 인구 수
temp = [[0]*n for _ in range(n)]
for _ in range(n):
    A.append(list(map(int,input().split())))
    
for x in range(n):
    for y in range(n):
        temp[x][y] = A[x][y] # 임시 인구 수 리스트
    
dx = [-1, 1, 0, 0]
dy = [0, 0, -1, 1]
lst = [] # dfs로 어디 방문했는지 리스트
result = 0
stack = [] # 최종 값 집어넣는 리스트

def dfs(x,y):
    global s, scnt
    for i in range(4): 
        nx = x +dx[i]
        ny = y +dy[i]
        if nx >=0 and nx < n and ny >= 0 and ny < n: # 범위 안에 있을때
            # 방문하지 않았고, 인구 수 차이가 조건을 충족 할 때
            if visited[nx][ny] == False and abs(temp[nx][ny] - temp[x][y]) >= l and abs(temp[nx][ny] - temp[x][y]) <= r:
                visited[nx][ny] = True
                lst.append((nx,ny)) # 리스트 안에 집어 넣고
                s += temp[nx][ny] # 인구 수 더하기
                scnt += 1 # 연합 나라 수 늘리기
                dfs(nx,ny)
                
                
while(True):
    # 하루 동안 일어나는 일
    visited = [[False]*n for _ in range(n)] # 하루동안 방문 여부 리스트 초기화
    for x in range(n):
        for y in range(n):
            s = temp[x][y] # 초기화
            scnt = 1 # 초기화
            lst.append((x,y))
            visited[x][y] = True
            dfs(x,y)
            if scnt > 1:
                nss = s // scnt
                for i,j in lst:
                    stack.append((i,j,nss)) # (x좌표, y좌표, 값) 형태로 집어넣기
            else: # scnt가 1이면
                lst.pop() # 방금 넣은 것 빼기
            lst = [] # 리스트 초기화 (한번의 bfs가 돌 때마다 초기화)
                
    for _ in range(len(stack)):
        a,b,c = stack.pop() # 스택에서 하나 빼고
        temp[a][b] = c
        
    if (A == temp) == True: # 차이가 없으면
        break
    else: # 원래 인구수와 바뀐 인구수의 차이가 있으면
        for x in range(n):
            for y in range(n):
                A[x][y] = temp[x][y]
                
    result += 1 # 인구 이동이 있었다
                
print(result)                

4 10 50
10 100 20 90
80 100 60 70
70 20 30 40
50 20 100 10
3


In [2]:
# 책 풀이, bfs 이용 
from collections import deque

# 땅의 크기 n, l, r 값 입력받기
n,l,r = map(int,input().split())

# 전체 나라의 정보를 입력 받기
graph = []
for _ in range(n):
    graph.append(list(map(int,input().split())))
    
dx = [-1, 0, 1, 0]
dy = [0, -1, 0, 1]
result = 0

# 특정 위치에서 출발하여 모든 연합을 체크한 뒤에 데이터 경신
def process(x,y,index):
    # (x,y)의 위치와 연결된 나라 (연합) 정보를 담는 리스트, 나중에 인구 수를 통합하기 위해
    united = []
    united.append((x,y)) 
    # 너비 우선 탐색(bfs)을 위한 큐 자료구조 정의
    q = deque()
    q.append((x,y))
    union[x][y] = index # 현재 연합의 번호 할당 , 방문 여부도 같이 확인 가능
    summary = graph[x][y] # 현재 연합의 전체 인구 수
    count = 1 # 현재 연합의 국가 수
    
    # 큐가 빌 때까지 반복
    while(q):
        x,y = q.popleft()
        # 현재 위치에서 4가지 방향을 확인하며
        for i in range(4):
            nx = x + dx[i]
            ny = y + dy[i]
            # 바로 옆에 나라를 확인하여
            if 0 <= nx < n and 0 <= ny < n and union[nx][ny] == -1: # 범위 안이고 번호가 할당 되지 않은 경우
                # 인구차이가 l이상 r이하라면
                if l <= abs(graph[nx][ny] - graph[x][y]) <= r:
                    q.append((nx,ny))
                    # 연합에 추가
                    union[nx][ny] = index
                    summary += graph[nx][ny]
                    count += 1
                    united.append((nx,ny))
                    
    # 연합 국가끼리 인구를 분배
    for i, j in united:
        graph[i][j] = summary // count
    return count

total_count = 0

# 더 이상 인구 이동을 할 수 없을 때까지 반복(하루 마다 반복)
while True:
    union = [[-1]*n for _ in range(n)] # 하루마다 연합 번호를 기록하는 리스트, 방문 여부 확인
    index = 0
    for i in range(n):
        for j in range(n):
            if union[i][j] == -1: # 해당 나라가 아직 처리되지 않았다면
                process(i,j,index)
                index += 1
       
    # 모든 인구 이동이 끝난 경우(index 값이 한 나라당 하나씩 주어진 경우 - 이웃 나라간 인구차이 범위에 없으면 이동 X )
    if index == n*n: 
        break
    total_count += 1
    
# 인구 이동 횟수 출력
print(total_count)


3 5 10
10 15 20
20 30 25
40 22 10
2
